In [7]:
!pip install pendulum

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 490.0/490.0 kB 45.8 MB/s eta 0:00:00


In [ ]:
#Import Library
import numpy as np
import pandas as pd
from datetime import datetime
import calendar 
import pendulum
from openpyxl import load_workbook

#Import Data
data_raw = pd.read_excel('2023-04-22_Data Sales FL Indonesia - Jan-MTD Apr.xlsx', sheet_name='Sheet1')
data_sku_product = pd.read_excel('SKU.xlsx', sheet_name='Product')
data_sku_sales = pd.read_excel('SKU.xlsx', sheet_name='Sales')
data_sku_inventory = pd.read_excel('SKU.xlsx', sheet_name='Inventory')

#Create Temporary List
rrp = []
crp = []
sales_site = []
date = []
week = []
store_name = []
division = []
subdiv_code = []
subdiv = []
gender = []
vendor_art = []
brand_name = []
franchise = []
store_type = []
color = []
month_week = []
month_month = []
final_rrp = []
final_rrp_new = []
sales_cost = []

#Missing Variables
missing_store_site = []
missing_article_product = []

#Filter Sales Quantity
data_raw = data_raw.drop(data_raw[data_raw['SalesQty'].isin([0])].index)  
data_raw = data_raw.reset_index(drop=True)

#Drop Duplicates But Keep Highest Value for SalesCost
data_sales_cost = data_raw[['Genarticle', 'SalesQty', 'SalesCost']].copy(deep=True)
data_sales_cost['Cost'] = np.where(data_sales_cost['SalesQty'] == 0, data_sales_cost['SalesCost'], data_sales_cost['SalesCost']/data_sales_cost['SalesQty'])
data_sales_cost = data_sales_cost.groupby('Genarticle', group_keys=False).apply(lambda x: x.loc[x.Cost.idxmax()])
data_sales_cost = data_sales_cost.reset_index(drop=True)

for i in range(len(data_raw)) : 
  #Delimit Date
  date.append(datetime.strptime(data_raw['SalesDate'][i], '%Y-%m-%d %H:%M:%S').strftime('%d/%m/%Y'))

  #Month Month
  num_month = date[i].split('/')[1]
  month_form = str(num_month).zfill(2) + " - " + calendar.month_abbr[int(num_month)]
  month_month.append(month_form)

  #Month Week
  week_month = pendulum.parse(datetime.strptime(date[i], '%d/%m/%Y').strftime("%Y-%m-%d"))
  if week_month.week_of_month == 5 :
    num_month = int(num_month) + 1
    month_week_form = str(num_month).zfill(2) + " - " + calendar.month_abbr[int(num_month)]
    month_week.append(month_week_form)
  else :
    month_week.append(month_form)

  #Week
  num_week = int(datetime.strptime(date[i], '%d/%m/%Y').date().strftime("%U"))
  if 49 <= num_week >= 52 : 
    num_week += 48
  else : 
    num_week -= 4
  num_week = str(num_week).zfill(2)
  week_name = "Wk " + num_week
  week.append(week_name)

  #Sales Site
  site_name = data_raw['Site'][i][0:2]
  if site_name=='FO' or site_name=='EF' :
    site_name = ' '
  else :
    site_name = 'Sales'
  sales_site.append(site_name)

  #CRP
  crp_num = int(float(data_raw['SalesQty'][i] * data_raw['CurrentPrice'][i]))
  crp.append(crp_num)

  #Store Name and Type
  site_loc = data_sku_sales.index[data_sku_sales['Site Code 1']==data_raw['Site'][i]].tolist()
  if len(site_loc)>0 :
    store_name.append(data_sku_sales['Desc 1'][site_loc[0]])
    store_type.append(data_sku_sales['Store Type'][site_loc[0]])
  else :
    site_loc = data_sku_inventory.index[data_sku_inventory['Site Code 2']==data_raw['Site'][i]].tolist()
    if len(site_loc)>0 :
      store_name.append(data_sku_inventory['Desc 2'][site_loc[0]])
      store_type.append(data_sku_inventory['Notes 3'][site_loc[0]])
    else : 
      missing_store_site.append(data_raw['Site'][i])
      store_name.append("#N/A")
      store_type.append("#N/A")
  
  #Division to Color
  #FL Indonesia
  genarticle_loc = data_sku_product.index[data_sku_product['Gen.art']==data_raw['Genarticle'][i]].tolist()
  
  #FL Philippines
  # genarticle_loc = data_sku_product.index[data_sku_product['Gen.art (PH)']==data_raw['Genarticle'][i]].tolist()
  
  if len(genarticle_loc)>0 :
    division.append(data_sku_product['FL Division'][genarticle_loc[0]])
    subdiv_code.append(data_sku_product['FL Subdivision Code'][genarticle_loc[0]])
    subdiv.append(data_sku_product['FL Subdivision'][genarticle_loc[0]])
    gender.append(data_sku_product['FL Gender / Age'][genarticle_loc[0]])
    vendor_art.append(data_sku_product['Vendor Art'][genarticle_loc[0]])
    brand_name.append(data_sku_product['Brand Name'][genarticle_loc[0]])
    franchise.append(data_sku_product['Franchise/Silhoutte'][genarticle_loc[0]])
    color.append(data_sku_product['Color Desc'][genarticle_loc[0]])
  else :
    missing_article_product.append(data_raw['Genarticle'][i])
    division.append("#N/A")
    subdiv_code.append("#N/A")
    subdiv.append("#N/A")
    gender.append("#N/A")
    vendor_art.append("#N/A")
    brand_name.append("#N/A")
    franchise.append("#N/A")
    color.append("#N/A")

  #SalesCost
  if len(str(int(float(data_raw['SalesCost'][i])))) <= 2 : 
    cost_loc = data_sales_cost.index[data_sales_cost['Genarticle']==data_raw['Genarticle'][i]].tolist()
    if len(cost_loc) > 0 : 
      if data_sales_cost['Cost'][cost_loc[0]] == 0 : 
        cost_loc = data_sku_product.index[data_sku_product['Gen.art']==data_raw['Genarticle'][i]].tolist()
        if len(cost_loc) > 0 : 
          sales_cost.append(int(float(data_raw['SalesQty'][i] * data_sku_product[' Est Landed Cost '][cost_loc[0]])))
        else : 
          missing_article_product.append(data_raw['Genarticle'][i])
          sales_cost.append("#N/A")
      else :   
        sales_cost.append(int(float(data_raw['SalesQty'][i] * data_sales_cost['Cost'][cost_loc[0]])))
    else : 
      missing_article_product.append(data_raw['Genarticle'][i])
      sales_cost.append("#N/A")
  else : 
    sales_cost.append(int(float(data_raw['SalesCost'][i])))

  #Final RRP and Discount
  check_price = data_raw['OriPrice'][i] - data_raw['CurrentPrice'][i]
  check_last_3_digits_op = str(int(float(data_raw['OriPrice'][i])))[-3:]
  check_last_3_digits_cp = str(int(float(data_raw['CurrentPrice'][i])))[-3:]

  if check_price >= 0 : 
    if check_last_3_digits_op == '000': 
      num_disc = (int(float(data_raw['OriPrice'][i])) - int(float(data_raw['CurrentPrice'][i]))) / int(float(data_raw['OriPrice'][i]))
      if num_disc > 0.8 : 
        final_rrp.append(int(float(data_raw['CurrentPrice'][i])))
      else : 
        final_rrp.append(int(float(data_raw['OriPrice'][i])))
    elif int(float(check_last_3_digits_op))%100 == 0 : 
      rrp_discount = int(float(data_raw['OriPrice'][i]/(1 - (int(float(check_last_3_digits_op))/1000))))
      if str(int(float(rrp_discount)))[-3:] == '999': 
        final_rrp.append(rrp_discount+1)
      elif str(int(float(rrp_discount)))[-3:] == '000': 
        final_rrp.append(rrp_discount)
      else : 
        price_loc = data_sku_product.index[data_sku_product['Gen.art']==data_raw['Genarticle'][i]].tolist()
        if len(price_loc) > 0 : 
          final_rrp.append(int(float(data_sku_product[' Retail Price '][price_loc[0]])))
        else : 
          missing_article_product.append(data_raw['Genarticle'][i])
          final_rrp.append("#N/A")
    else : 
      price_loc = data_sku_product.index[data_sku_product['Gen.art']==data_raw['Genarticle'][i]].tolist()
      if len(price_loc) > 0 : 
        final_rrp.append(int(float(data_sku_product[' Retail Price '][price_loc[0]])))
      else : 
        missing_article_product.append(data_raw['Genarticle'][i])
        final_rrp.append("#N/A")
  else : 
    if check_last_3_digits_cp == '000':
      final_rrp.append(int(float(data_raw['CurrentPrice'][i])))
    elif int(float(check_last_3_digits_cp))%100 == 0 :
      rrp_discount = int(float(data_raw['CurrentPrice'][i]/(1 - (int(float(check_last_3_digits_cp))/1000))))
      if str(int(float(rrp_discount)))[-3:] == '999': 
        final_rrp.append(rrp_discount+1)
      elif str(int(float(rrp_discount)))[-3:] == '000': 
        final_rrp.append(rrp_discount)
      else : 
        price_loc = data_sku_product.index[data_sku_product['Gen.art']==data_raw['Genarticle'][i]].tolist()
        if len(price_loc) > 0 : 
          final_rrp.append(int(float(data_sku_product[' Retail Price '][price_loc[0]])))
        else : 
          missing_article_product.append(data_raw['Genarticle'][i])
          final_rrp.append("#N/A")
    else : 
      price_loc = data_sku_product.index[data_sku_product['Gen.art']==data_raw['Genarticle'][i]].tolist()
      if len(price_loc)>0 :
        final_rrp.append(int(float(data_sku_product[' Retail Price '][price_loc[0]])))
      else :
        missing_article_product.append(data_raw['Genarticle'][i])
        final_rrp.append("#N/A")

#Combine All Except RRP
data_final = data_raw.copy(deep=True)
data_final['SalesCost'] = sales_cost
data_final['CRP'] = crp
data_final['Sales Site'] = sales_site
data_final['Date'] = date
data_final['Week'] = week
data_final['Store Name'] = store_name
data_final['FL Division'] = division
data_final['FL SubDiv Code'] = subdiv_code
data_final['FL SubDiv'] = subdiv
data_final['FL Gender / Age'] = gender
data_final['Vendor Art'] = vendor_art
data_final['BRAND'] = brand_name
data_final['Franchise (FTW)/Silhoutte (APP ACC)'] = franchise
data_final['Store Type'] = store_type
data_final['Color'] = color
data_final['Month-Week'] = month_week
data_final['Month-Month'] = month_month
data_final['New RRP'] = final_rrp

#Copy DataFrame
data_final_rrp_1 = data_final[['Genarticle', 'New RRP']].copy(deep=True)
data_final_rrp_1 = data_final_rrp_1.rename(columns={"New RRP": "NewRRP"})

#Drop Duplicates But Keep Highest Value for RRP
data_final_rrp_1 = data_final_rrp_1.groupby('Genarticle', group_keys=False).apply(lambda x: x.loc[x.NewRRP.idxmax()])
data_final_rrp_1 = data_final_rrp_1.reset_index(drop=True)

#Final RRP
for i in range(len(data_raw)) : 
  rrp_loc = data_final_rrp_1.index[data_final_rrp_1['Genarticle']==data_raw['Genarticle'][i]].tolist()
  if len(rrp_loc) > 0 : 
    final_rrp_new.append(data_final_rrp_1['NewRRP'][rrp_loc[0]]) 
    if str(int(float(data_final['New RRP'][i])))[-3:] != '000' : 
      data_final['New RRP'][i] = final_rrp_new[i]
    rrp.append(int(float(data_raw['SalesQty'][i] * data_final['New RRP'][i])))
  else : 
    final_rrp_new.append("#N/A") 
    rrp.append("#N/A")

#Combine All Remaining Column
data_final.insert(26, "RRP", rrp)
data_final['Max RRP in Each GenArticle'] = final_rrp_new

#Export Data
data_final.to_excel("Hasil 2023-04-22_Data Sales FL Indonesia - Jan-MTD Apr.xlsx", sheet_name = 'FINAL')

if len(missing_store_site) > 0:
  print("Terdapat Missing Store Site : %d Baris" % (len(missing_store_site)))
  missing_store_site = pd.DataFrame(missing_store_site, columns =['Store Site'])
  missing_store_site.to_excel('Missing Store Site.xlsx', sheet_name='Store Site Missing')
else : 
  print("Tidak Terdapat Missing Store Site : %d Baris" % (len(missing_store_site)))

if len(missing_article_product) > 0:
  print("Terdapat Missing Article Product : %d Baris" % (len(missing_article_product)))
  missing_article_product = pd.DataFrame(missing_article_product, columns =['Article Product'])
  missing_article_product.to_excel('Missing Article Product.xlsx', sheet_name='Article Product Missing')
else : 
  print("Tidak Missing Article Product : %d Baris" % (len(missing_article_product)))